# Project 7 - Notes for choosing hyperparameter for other variables

**Author: Linh Nguyen**<br>

**StudentID: 23161711**<br>

**Date: Sep 2022**

# 1. Load data and import package


In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Capstone_project

Mounted at /content/drive
/content/drive/MyDrive/Capstone_project


In [4]:
!pip3 install tensorflow_decision_forests --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 3.9 MB/s 
     |████████████████████████████████| 578.0 MB 13 kB/s 
     |████████████████████████████████| 1.7 MB 54.1 MB/s 
     |████████████████████████████████| 5.9 MB 64.7 MB/s 
     |████████████████████████████████| 438 kB 44.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [5]:
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras import layers
import seaborn as sns
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import  tensorflow as tf
from gensim.models import KeyedVectors
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

import tensorflow_decision_forests as tfdf

In [6]:
! ls /content/drive/MyDrive/Capstone_project/raw

diagnosis.csv  icd9toicd10cmgem.csv  pyxis.csv	 vitalsign.csv
edstays.csv    medrecon.csv	     triage.csv


# 2. Data cleansing


In [7]:
edstays = pd.read_csv('raw/edstays.csv')
edstays['y_var_adm_text'] = edstays['disposition']
edstays.loc[edstays['y_var_adm_text'] == 'TRANSFER', 'y_var_adm_text'] = 'ADMITTED'
edstays.loc[edstays['y_var_adm_text'] == 'ELOPED','y_var_adm_text'] = 'MISSING'
edstays.loc[edstays['y_var_adm_text'] == 'LEFT WITHOUT BEING SEEN','y_var_adm_text'] = 'MISSING'
edstays.loc[edstays['y_var_adm_text'] == 'OTHER','y_var_adm_text'] = 'MISSING'
edstays.loc[edstays['y_var_adm_text'] == 'LEFT AGAINST MEDICAL ADVICE','y_var_adm_text'] = 'MISSING'
edstays.loc[edstays['y_var_adm_text'] == 'EXPIRED','y_var_adm_text'] = 'MISSING'
edstays_admit = edstays[edstays['y_var_adm_text'] !='MISSING'].reset_index(drop = True)
edstays_admit['key'] = edstays_admit['subject_id'].astype(str)+'_'+edstays_admit['stay_id'].astype(str)

In [8]:
edstays_admit.loc[edstays_admit['race'].str.contains('WHITE'), 'race_color'] = 'WHITE'
edstays_admit.loc[edstays_admit['race'].str.contains('BLACK'), 'race_color'] = 'BLACK'
edstays_admit.loc[edstays_admit['race'].str.contains('ASIAN'), 'race_color'] = 'ASIAN'
edstays_admit.loc[edstays_admit['race'].str.contains('LATINO'), 'race_color'] = 'HISPANIC/LATINO'
edstays_admit.loc[edstays_admit['race'].str.contains('HISPANIC'), 'race_color'] = 'HISPANIC/LATINO'
edstays_admit.loc[edstays_admit['race_color'].isna(), 'race_color'] = 'OTHER'

In [9]:
edstays_admit['DateTime_in'] = pd.to_datetime(edstays_admit['intime'], format='%d/%m/%Y %H:%M')
edstays_admit['DateTime_out'] = pd.to_datetime(edstays_admit['outtime'], format='%d/%m/%Y %H:%M')

In [10]:
edstays_admit['stay'] = edstays_admit['DateTime_out'] - edstays_admit['DateTime_in']

In [11]:
edstays_admit_1 = edstays_admit.sort_values(by = ['subject_id','DateTime_in'], ascending=True).reset_index(drop=True)

In [12]:
edstay_hist = {}
for i in range(len(edstays_admit_1)):
    if i == 0:
        edstay_hist[i] = 'NO HISTORY'
    else:
        if edstays_admit_1['subject_id'][i] == edstays_admit_1['subject_id'][i-1]:
            edstay_hist[i] = edstays_admit_1['y_var_adm_text'][i-1]
        else:
            edstay_hist[i] = 'NO HISTORY'

In [13]:
edstay_laststay = {}
for i in range(len(edstays_admit_1)):
    if i == 0:
        edstay_laststay[i] = np.timedelta64(0, 'D')
    else:
        if edstays_admit_1['subject_id'][i] == edstays_admit_1['subject_id'][i-1]: 
          edstay_laststay[i] = edstays_admit_1['stay'][i-1]+edstay_laststay[i-1]
        else:
            edstay_laststay[i] = np.timedelta64(0, 'D')

In [14]:
df1 = pd.DataFrame.from_dict(edstay_hist,orient='index')
df1.columns = ['historical_stay_status']
edstays_admit_1 = pd.merge(edstays_admit_1,df1,how = 'left',left_index=True,right_index=True) 

In [15]:
df1 = pd.DataFrame.from_dict(edstay_laststay,orient='index')
df1.columns = ['historical_stay_length']
edstays_admit_1 = pd.merge(edstays_admit_1,df1,how = 'left',left_index=True,right_index=True) 

In [16]:
edstays_admit_1['historical_stay_length_in_day'] = edstays_admit_1['historical_stay_length']/(np.timedelta64(1, 'D'))

In [17]:
edstays_admit_2 = pd.merge(edstays_admit,edstays_admit_1[['historical_stay_status','historical_stay_length','historical_stay_length_in_day','subject_id','stay_id']],how = 'left',on=['subject_id', 'stay_id'])

In [18]:
edstays_admit_2.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'gender',
       'race', 'arrival_transport', 'disposition', 'y_var_adm_text', 'key',
       'race_color', 'DateTime_in', 'DateTime_out', 'stay',
       'historical_stay_status', 'historical_stay_length',
       'historical_stay_length_in_day'],
      dtype='object')

In [19]:
triage = pd.read_csv('raw/triage.csv')


In [20]:
edstay_ad = pd.merge(edstays_admit_2,triage,how = 'left',on=['subject_id', 'stay_id'])

In [21]:
edstay_ad['y_var'] = 0
edstay_ad.loc[edstay_ad['y_var_adm_text'] == 'ADMITTED', 'y_var']  = 1

In [22]:
edstay_ad.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'intime', 'outtime', 'gender',
       'race', 'arrival_transport', 'disposition', 'y_var_adm_text', 'key',
       'race_color', 'DateTime_in', 'DateTime_out', 'stay',
       'historical_stay_status', 'historical_stay_length',
       'historical_stay_length_in_day', 'temperature', 'heartrate', 'resprate',
       'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'chiefcomplaint', 'y_var'],
      dtype='object')

In [23]:
edstay_ad = edstay_ad.drop(['subject_id','hadm_id','stay_id','intime','outtime','race', 'disposition','y_var_adm_text','key', 'DateTime_in', 'DateTime_out','stay','historical_stay_length','chiefcomplaint', 'pain'], axis = 1)

In [24]:
edstay_ad

,gender,arrival_transport,race_color,historical_stay_status,historical_stay_length_in_day,temperature,heartrate,resprate,o2sat,sbp,dbp,acuity,y_var
0,F,AMBULANCE,WHITE,NO HISTORY,0.000000,98.4,70.0,16.0,97.0,106.0,63.0,3.0,1
1,F,AMBULANCE,WHITE,ADMITTED,0.175694,98.9,88.0,18.0,97.0,116.0,88.0,3.0,1
2,F,AMBULANCE,WHITE,ADMITTED,1.309722,99.4,105.0,18.0,96.0,106.0,57.0,3.0,1
3,F,AMBULANCE,WHITE,ADMITTED,0.409722,97.8,87.0,14.0,97.0,71.0,43.0,2.0,0
4,F,AMBULANCE,WHITE,HOME,0.972222,98.7,77.0,16.0,98.0,96.0,50.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
428279,M,WALK IN,BLACK,ADMITTED,0.551389,98.8,92.0,18.0,100.0,122.0,77.0,3.0,1
428280,M,WALK IN,BLACK,NO HISTORY,0.000000,98.6,80.0,18.0,100.0,161.0,100.0,3.0,1
428281,F,AMBULANCE,WHITE,ADMITTED,1.011111,96.6,112.0,18.0,100.0,110.0,82.0,2.0,1
428282,F,WALK IN,WHITE,NO HISTORY,0.000000,98.1,83.0,18.0,100.0,107.0,75.0,2.0,1


In [25]:
edstay_ad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428284 entries, 0 to 428283
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   gender                         428284 non-null  object 
 1   arrival_transport              428284 non-null  object 
 2   race_color                     428284 non-null  object 
 3   historical_stay_status         428284 non-null  object 
 4   historical_stay_length_in_day  428284 non-null  float64
 5   temperature                    405762 non-null  float64
 6   heartrate                      412132 non-null  float64
 7   resprate                       408822 non-null  float64
 8   o2sat                          408594 non-null  float64
 9   sbp                            410935 non-null  float64
 10  dbp                            410124 non-null  float64
 11  acuity                         422216 non-null  float64
 12  y_var                         

In [26]:
updated_edstay_ad = edstay_ad
updated_edstay_ad['temperaturemissing'] = updated_edstay_ad['temperature'].isnull().astype(int)
updated_edstay_ad['heartratemissing'] = updated_edstay_ad['heartrate'].isnull().astype(int)
updated_edstay_ad['respratemissing'] = updated_edstay_ad['resprate'].isnull().astype(int)
updated_edstay_ad['o2satmissing'] = updated_edstay_ad['o2sat'].isnull().astype(int)
updated_edstay_ad['sbpmissing'] = updated_edstay_ad['sbp'].isnull().astype(int)
updated_edstay_ad['dbpmissing'] = updated_edstay_ad['dbp'].isnull().astype(int)
updated_edstay_ad['acuitymissing'] = updated_edstay_ad['acuity'].isnull().astype(int)

In [27]:
num_attr = edstay_ad.select_dtypes(include=['float64']).columns
cat_attr = edstay_ad.select_dtypes(include=['O']).columns
#bool_attr = edstay_ad.select_dtypes(include=['bool']).columns

In [28]:
my_imputer = SimpleImputer(strategy = 'median')
updated_edstay_ad[num_attr] = my_imputer.fit_transform(updated_edstay_ad[num_attr])
updated_edstay_ad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 428284 entries, 0 to 428283
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   gender                         428284 non-null  object 
 1   arrival_transport              428284 non-null  object 
 2   race_color                     428284 non-null  object 
 3   historical_stay_status         428284 non-null  object 
 4   historical_stay_length_in_day  428284 non-null  float64
 5   temperature                    428284 non-null  float64
 6   heartrate                      428284 non-null  float64
 7   resprate                       428284 non-null  float64
 8   o2sat                          428284 non-null  float64
 9   sbp                            428284 non-null  float64
 10  dbp                            428284 non-null  float64
 11  acuity                         428284 non-null  float64
 12  y_var                         

# 3. Split dataset

In [29]:
#set random index, create train set and validation set from the original train set

#create array of random index
np.random.seed(123)
ind = np.arange(len(updated_edstay_ad))
np.random.shuffle(ind)
#create testing index  = 0 to 0.20 * random index
test_index = ind[:int(len(updated_edstay_ad) * 0.20)]
#create train index = 0.20 * random index to end
train_index = ind[int(len(updated_edstay_ad) * 0.20):]

In [30]:
X_train = updated_edstay_ad.drop('y_var',axis = 1).iloc[train_index]
y_train = updated_edstay_ad['y_var'].iloc[train_index]
X_test = updated_edstay_ad.drop('y_var',axis = 1).iloc[test_index]
y_test = updated_edstay_ad['y_var'].iloc[test_index]
Train = updated_edstay_ad.iloc[train_index]
Test = updated_edstay_ad.iloc[test_index]

5. Transform dataset

In [31]:
trans_pip = ColumnTransformer([
    ("num",StandardScaler(), num_attr),
    ("cat",OneHotEncoder(),cat_attr)
],remainder='passthrough')



# 4. Random forest

In [32]:
!pip install --quiet optuna

     |████████████████████████████████| 348 kB 4.2 MB/s 
     |████████████████████████████████| 81 kB 10.0 MB/s 
     |████████████████████████████████| 209 kB 72.2 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 147 kB 73.8 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 112 kB 77.7 MB/s 


In [33]:
RANDOM_SEED = 42
from sklearn.model_selection import KFold
import optuna

# 5-fold CV
kfolds = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
# Define the helper function so that it can be reused
def tune(objective):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)

    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}\n")
    print(f"Optimized parameters: {params}\n")
    return params


In [ ]:
def RF(trial):
    num_trees = trial.suggest_int("units", 200, 300)
    min_examples = trial.suggest_int("min_examples", 100, 500)
    max_depth = trial.suggest_int("max_depth", 10, 25)


    model = tfdf.keras.RandomForestModel(num_trees = num_trees, min_examples = min_examples, max_depth = max_depth)
    weights = {0:1-(sum(Train['y_var'])/len(Train)),1:sum(Train['y_var'])/len(Train)}

    model.compile(metrics=[keras.metrics.Accuracy(name="accuracy")])

    acc_per_fold = []
    loss_per_fold = []
    for train, test in kfolds.split(Train):
        train_trans = tfdf.keras.pd_dataframe_to_tf_dataset(pd.DataFrame(trans_pip.fit_transform(Train.iloc[train]), columns=trans_pip.get_feature_names_out()), label="remainder__y_var")
        test_trans = tfdf.keras.pd_dataframe_to_tf_dataset(pd.DataFrame(trans_pip.transform(Train.iloc[test]), columns=trans_pip.get_feature_names_out()), label="remainder__y_var")
        model.fit(train_trans, validation_data = test_trans, batch_size = 256, class_weight = weights)

  # Generate generalization metrics
        scores = model.evaluate(test_trans)
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
    
    return np.mean(acc_per_fold)

RF_params = tune(RF)

[I 2022-09-13 17:50:23,115] A new study created in memory with name: no-name-4972f4e4-8a2e-4ea1-bdc9-6b6fe4cb3eca


Use /tmp/tmpkso9a6mf as temporary training directory


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


Reading training dataset...
Training dataset read in 0:00:06.599483. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.069293. Found 68526 examples.
Training model...
Model trained in 0:05:56.853944
Compiling model...


...

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


Reading training dataset...
Training dataset read in 0:00:04.215455. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.107405. Found 68525 examples.
Training model...
Model trained in 0:05:07.584924
Compiling model...
Model compiled.
69/69 [==============================] - 3s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4054


[I 2022-09-13 22:52:41,073] Trial 9 finished with value: 40.43452501296997 and parameters: {'units': 240, 'min_examples': 500, 'max_depth': 19}. Best is trial 8 with value: 45.17844498157501.


Best score: 45.17844498157501

Optimized parameters: {'units': 280, 'min_examples': 288, 'max_depth': 17}



[I 2022-09-13 17:50:23,115] A new study created in memory with name: no-name-4972f4e4-8a2e-4ea1-bdc9-6b6fe4cb3eca
Use /tmp/tmpkso9a6mf as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.599483. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.069293. Found 68526 examples.
Training model...
Model trained in 0:05:56.853944
Compiling model...
WARNING:tensorflow:5 out of the last 90 calls to <function InferenceCoreModel.make_predict_function.<locals>.predict_function_trained at 0x7fe2b91c9830> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Model compiled.
69/69 [==============================] - 3s 46ms/step - loss: 0.0000e+00 - accuracy: 0.4003
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.480193. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:05.461918. Found 68526 examples.
Training model...
Model trained in 0:05:52.069625
Compiling model...
WARNING:tensorflow:6 out of the last 91 calls to <function InferenceCoreModel.make_predict_function.<locals>.predict_function_trained at 0x7fe2b92c7680> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Model compiled.
69/69 [==============================] - 4s 60ms/step - loss: 0.0000e+00 - accuracy: 0.3963
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:07.412832. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:04.559198. Found 68526 examples.
Training model...
Model trained in 0:05:50.167847
Compiling model...
Model compiled.
69/69 [==============================] - 3s 44ms/step - loss: 0.0000e+00 - accuracy: 0.4010
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.653613. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.666928. Found 68525 examples.
Training model...
Model trained in 0:05:55.879337
Compiling model...
Model compiled.
69/69 [==============================] - 3s 44ms/step - loss: 0.0000e+00 - accuracy: 0.4025
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.483966. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.594007. Found 68525 examples.
Training model...
Model trained in 0:05:52.721046
Compiling model...
Model compiled.
69/69 [==============================] - 3s 45ms/step - loss: 0.0000e+00 - accuracy: 0.4024
[I 2022-09-13 18:22:18,576] Trial 0 finished with value: 40.051018595695496 and parameters: {'units': 254, 'min_examples': 425, 'max_depth': 25}. Best is trial 0 with value: 40.051018595695496.
Use /tmp/tmpo5vo0ehy as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.646685. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.074908. Found 68526 examples.
Training model...
Model trained in 0:04:52.773646
Compiling model...
Model compiled.
69/69 [==============================] - 3s 41ms/step - loss: 0.0000e+00 - accuracy: 0.3693
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.299432. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.582782. Found 68526 examples.
Training model...
Model trained in 0:04:55.318507
Compiling model...
Model compiled.
69/69 [==============================] - 3s 42ms/step - loss: 0.0000e+00 - accuracy: 0.3659
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.487600. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:05.403798. Found 68526 examples.
Training model...
Model trained in 0:04:50.948906
Compiling model...
Model compiled.
69/69 [==============================] - 3s 42ms/step - loss: 0.0000e+00 - accuracy: 0.3705
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.412576. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.637866. Found 68525 examples.
Training model...
Model trained in 0:04:53.264220
Compiling model...
Model compiled.
69/69 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - accuracy: 0.3710
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.487537. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.679609. Found 68525 examples.
Training model...
Model trained in 0:04:56.469248
Compiling model...
Model compiled.
69/69 [==============================] - 3s 42ms/step - loss: 0.0000e+00 - accuracy: 0.3726
[I 2022-09-13 18:49:32,846] Trial 1 finished with value: 36.98909640312195 and parameters: {'units': 292, 'min_examples': 105, 'max_depth': 12}. Best is trial 0 with value: 40.051018595695496.
Use /tmp/tmpto_e_n9f as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.439880. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.269033. Found 68526 examples.
Training model...
Model trained in 0:05:23.942494
Compiling model...
Model compiled.
69/69 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - accuracy: 0.3959
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.443713. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:05.548901. Found 68526 examples.
Training model...
Model trained in 0:05:23.444981
Compiling model...
Model compiled.
69/69 [==============================] - 3s 42ms/step - loss: 0.0000e+00 - accuracy: 0.3905
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.791952. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.092312. Found 68526 examples.
Training model...
Model trained in 0:05:19.121207
Compiling model...
Model compiled.
69/69 [==============================] - 3s 41ms/step - loss: 0.0000e+00 - accuracy: 0.3936
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.137987. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:05.755951. Found 68525 examples.
Training model...
Model trained in 0:05:24.312278
Compiling model...
Model compiled.
69/69 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - accuracy: 0.3960
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.958450. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.487263. Found 68525 examples.
Training model...
Model trained in 0:05:25.565532
Compiling model...
Model compiled.
69/69 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - accuracy: 0.3935
[I 2022-09-13 19:20:48,373] Trial 2 finished with value: 39.38907504081726 and parameters: {'units': 262, 'min_examples': 304, 'max_depth': 14}. Best is trial 0 with value: 40.051018595695496.
Use /tmp/tmpdcdv4_xj as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.451772. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:06.168622. Found 68526 examples.
Training model...
Model trained in 0:06:30.406122
Compiling model...
Model compiled.
69/69 [==============================] - 4s 57ms/step - loss: 0.0000e+00 - accuracy: 0.4387
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:08.212446. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.640116. Found 68526 examples.
Training model...
Model trained in 0:06:33.822040
Compiling model...
Model compiled.
69/69 [==============================] - 4s 58ms/step - loss: 0.0000e+00 - accuracy: 0.4355
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.258449. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:05.621827. Found 68526 examples.
Training model...
Model trained in 0:06:34.937345
Compiling model...
Model compiled.
69/69 [==============================] - 4s 57ms/step - loss: 0.0000e+00 - accuracy: 0.4392
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.857141. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.499518. Found 68525 examples.
Training model...
Model trained in 0:06:33.080409
Compiling model...
Model compiled.
69/69 [==============================] - 4s 58ms/step - loss: 0.0000e+00 - accuracy: 0.4376
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.861302. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.568327. Found 68525 examples.
Training model...
Model trained in 0:06:31.392820
Compiling model...
Model compiled.
69/69 [==============================] - 4s 52ms/step - loss: 0.0000e+00 - accuracy: 0.4428
[I 2022-09-13 19:58:04,822] Trial 3 finished with value: 43.876157999038696 and parameters: {'units': 295, 'min_examples': 215, 'max_depth': 15}. Best is trial 3 with value: 43.876157999038696.
Use /tmp/tmpbbane4zl as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.159476. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.116464. Found 68526 examples.
Training model...
Model trained in 0:04:52.987752
Compiling model...
Model compiled.
69/69 [==============================] - 3s 38ms/step - loss: 0.0000e+00 - accuracy: 0.3617
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.967696. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.255876. Found 68526 examples.
Training model...
Model trained in 0:04:57.160337
Compiling model...
Model compiled.
69/69 [==============================] - 3s 41ms/step - loss: 0.0000e+00 - accuracy: 0.3569
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.987268. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.445666. Found 68526 examples.
Training model...
Model trained in 0:04:55.324206
Compiling model...
Model compiled.
69/69 [==============================] - 3s 40ms/step - loss: 0.0000e+00 - accuracy: 0.3605
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.464525. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.829637. Found 68525 examples.
Training model...
Model trained in 0:04:54.857231
Compiling model...
Model compiled.
69/69 [==============================] - 3s 39ms/step - loss: 0.0000e+00 - accuracy: 0.3591
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.512144. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.151769. Found 68525 examples.
Training model...
Model trained in 0:04:51.580699
Compiling model...
Model compiled.
69/69 [==============================] - 3s 45ms/step - loss: 0.0000e+00 - accuracy: 0.3624
[I 2022-09-13 20:25:19,322] Trial 4 finished with value: 36.011067032814026 and parameters: {'units': 243, 'min_examples': 115, 'max_depth': 14}. Best is trial 3 with value: 43.876157999038696.
Use /tmp/tmp7yf7hon_ as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:06.652820. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.051464. Found 68526 examples.
Training model...
Model trained in 0:05:35.456552
Compiling model...
Model compiled.
69/69 [==============================] - 3s 43ms/step - loss: 0.0000e+00 - accuracy: 0.4197
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.606725. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.758467. Found 68526 examples.
Training model...
Model trained in 0:05:26.182827
Compiling model...
Model compiled.
69/69 [==============================] - 3s 44ms/step - loss: 0.0000e+00 - accuracy: 0.4156
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.665068. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.208143. Found 68526 examples.
Training model...
Model trained in 0:05:25.613505
Compiling model...
Model compiled.
69/69 [==============================] - 3s 44ms/step - loss: 0.0000e+00 - accuracy: 0.4179
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.619363. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.791179. Found 68525 examples.
Training model...
Model trained in 0:05:26.560665
Compiling model...
Model compiled.
69/69 [==============================] - 3s 44ms/step - loss: 0.0000e+00 - accuracy: 0.4229
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.655970. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.205903. Found 68525 examples.
Training model...
Model trained in 0:05:26.411272
Compiling model...
Model compiled.
69/69 [==============================] - 3s 46ms/step - loss: 0.0000e+00 - accuracy: 0.4228
[I 2022-09-13 20:55:24,616] Trial 5 finished with value: 41.978474855422974 and parameters: {'units': 238, 'min_examples': 155, 'max_depth': 16}. Best is trial 3 with value: 43.876157999038696.
Use /tmp/tmpl147puct as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:05.098366. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.116877. Found 68526 examples.
Training model...
Model trained in 0:06:48.105021
Compiling model...
Model compiled.
69/69 [==============================] - 4s 54ms/step - loss: 0.0000e+00 - accuracy: 0.3904
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.493795. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.161015. Found 68526 examples.
Training model...
Model trained in 0:06:47.003002
Compiling model...
Model compiled.
69/69 [==============================] - 4s 54ms/step - loss: 0.0000e+00 - accuracy: 0.3916
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.488887. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.133387. Found 68526 examples.
Training model...
Model trained in 0:06:45.637261
Compiling model...
Model compiled.
69/69 [==============================] - 4s 53ms/step - loss: 0.0000e+00 - accuracy: 0.3916
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.673883. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.659257. Found 68525 examples.
Training model...
Model trained in 0:06:48.780673
Compiling model...
Model compiled.
69/69 [==============================] - 4s 54ms/step - loss: 0.0000e+00 - accuracy: 0.3948
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.474915. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.173522. Found 68525 examples.
Training model...
Model trained in 0:06:49.318714
Compiling model...
Model compiled.
69/69 [==============================] - 4s 55ms/step - loss: 0.0000e+00 - accuracy: 0.3924
[I 2022-09-13 21:31:36,877] Trial 6 finished with value: 39.21833634376526 and parameters: {'units': 294, 'min_examples': 312, 'max_depth': 23}. Best is trial 3 with value: 43.876157999038696.
Use /tmp/tmp7pxy2379 as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.680362. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.504313. Found 68526 examples.
Training model...
Model trained in 0:04:07.162767
Compiling model...
Model compiled.
69/69 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.4382
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.322452. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.087162. Found 68526 examples.
Training model...
Model trained in 0:04:12.353642
Compiling model...
Model compiled.
69/69 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.4376
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.301773. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.084898. Found 68526 examples.
Training model...
Model trained in 0:04:10.145872
Compiling model...
Model compiled.
69/69 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.4447
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.347791. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.123920. Found 68525 examples.
Training model...
Model trained in 0:04:09.771190
Compiling model...
Model compiled.
69/69 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.4462
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.304702. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.114776. Found 68525 examples.
Training model...
Model trained in 0:04:11.985436
Compiling model...
Model compiled.
69/69 [==============================] - 2s 33ms/step - loss: 0.0000e+00 - accuracy: 0.4361
[I 2022-09-13 21:53:37,110] Trial 7 finished with value: 44.05477702617645 and parameters: {'units': 258, 'min_examples': 104, 'max_depth': 12}. Best is trial 7 with value: 44.05477702617645.
Use /tmp/tmpl2spq7wd as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.618344. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.469071. Found 68526 examples.
Training model...
Model trained in 0:06:12.528949
Compiling model...
Model compiled.
69/69 [==============================] - 4s 60ms/step - loss: 0.0000e+00 - accuracy: 0.4491
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.241738. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.030780. Found 68526 examples.
Training model...
Model trained in 0:06:14.694768
Compiling model...
Model compiled.
69/69 [==============================] - 3s 47ms/step - loss: 0.0000e+00 - accuracy: 0.4472
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.305622. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.958766. Found 68526 examples.
Training model...
Model trained in 0:06:16.347041
Compiling model...
Model compiled.
69/69 [==============================] - 3s 48ms/step - loss: 0.0000e+00 - accuracy: 0.4545
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.267350. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.108030. Found 68525 examples.
Training model...
Model trained in 0:06:14.959444
Compiling model...
Model compiled.
69/69 [==============================] - 3s 47ms/step - loss: 0.0000e+00 - accuracy: 0.4532
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.243247. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.126217. Found 68525 examples.
Training model...
Model trained in 0:06:17.100202
Compiling model...
Model compiled.
69/69 [==============================] - 3s 48ms/step - loss: 0.0000e+00 - accuracy: 0.4550
[I 2022-09-13 22:26:09,662] Trial 8 finished with value: 45.17844498157501 and parameters: {'units': 280, 'min_examples': 288, 'max_depth': 17}. Best is trial 8 with value: 45.17844498157501.
Use /tmp/tmpa2wiehs4 as temporary training directory
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.588924. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:02.525921. Found 68526 examples.
Training model...
Model trained in 0:05:02.796644
Compiling model...
Model compiled.
69/69 [==============================] - 3s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4028
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.292652. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.118427. Found 68526 examples.
Training model...
Model trained in 0:05:02.454026
Compiling model...
Model compiled.
69/69 [==============================] - 3s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4028
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.276920. Found 274102 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68526, shape=(), dtype=int32)
Validation dataset read in 0:00:01.071659. Found 68526 examples.
Training model...
Model trained in 0:05:02.571354
Compiling model...
Model compiled.
69/69 [==============================] - 3s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4043
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.287953. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.097540. Found 68525 examples.
Training model...
Model trained in 0:05:02.369890
Compiling model...
Model compiled.
69/69 [==============================] - 3s 36ms/step - loss: 0.0000e+00 - accuracy: 0.4065
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
Warning: Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
WARNING:absl:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.
Warning: Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
WARNING:absl:Model constructor argument batch_size=256 not supported. See https://www.tensorflow.org/decision_forests/migration for an explanation about the specificities of TF-DF.
Reading training dataset...
Training dataset read in 0:00:04.215455. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:01.107405. Found 68525 examples.
Training model...
Model trained in 0:05:07.584924
Compiling model...
Model compiled.
69/69 [==============================] - 3s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4054
[I 2022-09-13 22:52:41,073] Trial 9 finished with value: 40.43452501296997 and parameters: {'units': 240, 'min_examples': 500, 'max_depth': 19}. Best is trial 8 with value: 45.17844498157501.
Best score: 45.17844498157501

Optimized parameters: {'units': 280, 'min_examples': 288, 'max_depth': 17}

In [39]:
#create array of random index
np.random.seed(1)
ind1 = np.arange(len(Train))
np.random.shuffle(ind1)
#create testing index  = 0 to 0.20 * random index
val_index = ind1[:int(len(Train) * 0.20)]
#create train index = 0.20 * random index to end
train_index1 = ind1[int(len(Train) * 0.20):]

In [55]:
model = tfdf.keras.RandomForestModel(num_trees = 300, min_examples = 280, max_depth = 17)
weights = {0:1-(sum(Train['y_var'])/len(Train)),1:sum(Train['y_var'])/len(Train)}
model.compile(metrics=[keras.metrics.Accuracy(name="accuracy")])

train_trans = tfdf.keras.pd_dataframe_to_tf_dataset(pd.DataFrame(trans_pip.fit_transform(Train.iloc[train_index1]), columns=trans_pip.get_feature_names_out()), label="remainder__y_var")
val_trans = tfdf.keras.pd_dataframe_to_tf_dataset(pd.DataFrame(trans_pip.transform(Train.iloc[val_index]), columns=trans_pip.get_feature_names_out()), label="remainder__y_var")

model.fit(train_trans,validation_data = val_trans,  batch_size = 256, class_weight = weights)
#


Use /tmp/tmpgko2aqjd as temporary training directory


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


Reading training dataset...


Training dataset read in 0:00:05.155448. Found 274103 examples.
Reading validation dataset...
Num validation examples: tf.Tensor(68525, shape=(), dtype=int32)
Validation dataset read in 0:00:02.102428. Found 68525 examples.
Training model...
Model trained in 0:01:35.938049
Compiling model...


Model compiled.


In [52]:
inspector = model.make_inspector()
print(inspector.num_trees())
print(inspector.variable_importances())

300
{'MEAN_MIN_DEPTH': [("cat__arrival_transport_HELICOPTER" (1; #1), 9.790373801599987), ("cat__arrival_transport_OTHER" (1; #2), 9.790373801599987), ("__LABEL" (4; #30), 9.790373801599987), ("__WEIGHTS" (1; #31), 9.790373801599987), ("cat__race_color_ASIAN" (1; #10), 9.770904292754), ("cat__race_color_OTHER" (1; #13), 9.693651466681818), ("cat__race_color_HISPANIC/LATINO" (1; #12), 9.63384911163411), ("cat__gender_F" (1; #5), 9.58740425778652), ("cat__gender_M" (1; #6), 9.570388877968956), ("remainder__o2satmissing" (1; #26), 9.468817852590485), ("remainder__sbpmissing" (1; #28), 9.467572265019607), ("remainder__acuitymissing" (1; #23), 9.445217402768515), ("remainder__dbpmissing" (1; #24), 9.422287809817991), ("remainder__temperaturemissing" (1; #29), 9.272321089378318), ("cat__historical_stay_status_NO_HISTORY" (1; #9), 9.173604750326326), ("cat__race_color_BLACK" (1; #11), 9.109139026313773), ("remainder__respratemissing" (1; #27), 9.101159055028354), ("cat__arrival_transport_UNKN

In [53]:
test_trans = tfdf.keras.pd_dataframe_to_tf_dataset(pd.DataFrame(trans_pip.transform(Test), columns=trans_pip.get_feature_names_out()), label="remainder__y_var")
result = model.evaluate(test_trans)

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core_inference.py:873: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


86/86 [==============================] - 3s 32ms/step - loss: 0.0000e+00 - accu(racy: 0.3973


In [54]:
model.evaluate(train_trans)

275/275 [==============================] - 9s 33ms/step - loss: 0.0000e+00 - accu(racy: 0.3988


[0.0, 0.3987807631492615]